In [1]:
import findspark
findspark.init()
import pyspark
from operator import add
from pyspark import SparkConf
from pyspark.ml.feature import NGram
from pyspark.sql.functions import col,udf
from pyspark.sql import SQLContext

In [2]:
sc = pyspark.SparkContext('local[*]',appName="DocClassification")
sqlc = SQLContext(sc)

In [3]:
rdd1= sc.wholeTextFiles("/home/vyom/UGA/DSP/Project2/data/train/asm/0bN6ODYWw2xeCQBn3tEg.asm,/home/vyom/UGA/DSP/Project2/data/train/asm/0eN9lyQfwmTVk7C2ZoYp.asm,/home/vyom/UGA/DSP/Project2/data/train/asm/0H63jydvIahOVqgx5Kfo.asm,/home/vyom/UGA/DSP/Project2/data/train/asm/0hZEqJ5eMVjU21HAG7Ii.asm")
#rdd1= sc.wholeTextFiles("/home/vyom/UGA/DSP/Project2/data/train/asm/")

In [4]:
rdd1.count()

4

In [5]:
opcodes= sc.broadcast(sc.textFile("/home/vyom/UGA/DSP/Project2/allOpcodes.txt").collect())

In [6]:
len(opcodes.value)

2397

# Get Opcodes list using python approach

In [7]:
rdd2 = rdd1.map(lambda x: x[1].split()).map(lambda x: [word for word in x if word in opcodes.value])

In [8]:
rdd2.collect()

[['in',
  'mov',
  'jmp',
  'int',
  'push',
  'mov',
  'mov',
  'call',
  'test',
  'jz',
  'push',
  'call',
  'add',
  'mov',
  'pop',
  'retn',
  'xor',
  'retn',
  'push',
  'mov',
  'mov',
  'test',
  'jz',
  'mov',
  'mov',
  'push',
  'push',
  'call',
  'mov',
  'mov',
  'mov',
  'add',
  'shl',
  'add',
  'mov',
  'add',
  'mov',
  'mov',
  'mov',
  'mov',
  'mov',
  'pop',
  'retn',
  'mov',
  'mov',
  'not',
  'not',
  'mov',
  'mov',
  'mov',
  'mov',
  'mov',
  'lea',
  'mov',
  'mov',
  'mov',
  'mov',
  'lea',
  'mov',
  'mov',
  'retn',
  'int',
  'mov',
  'sub',
  'test',
  'ja',
  'xor',
  'lea',
  'push',
  'call',
  'add',
  'add',
  'retn',
  'or',
  'xor',
  'div',
  'cmp',
  'jnb',
  'lea',
  'push',
  'lea',
  'mov',
  'call',
  'push',
  'lea',
  'push',
  'mov',
  'call',
  'int',
  'mov',
  'sub',
  'test',
  'ja',
  'xor',
  'push',
  'call',
  'add',
  'add',
  'retn',
  'or',
  'xor',
  'div',
  'cmp',
  'jnb',
  'lea',
  'push',
  'lea',
  'mov',
  'call

# Get Opcodes list using mostly spark

In [7]:
opcodesInDoc = rdd1.zipWithIndex().map(lambda x: (x[1],x[0][1].split())).flatMapValues(lambda x: x).filter(lambda x: x[1] in opcodes.value).groupByKey().map(lambda x: (x[0],list(x[1])))

In [8]:
#opcodesInDoc.collect()

# Get N-grams and N-grams count

In [9]:
ngramFrame = sqlc.createDataFrame(opcodesInDoc,["docId","opcodes"])

In [10]:
twoGram = NGram(n=2, inputCol="opcodes", outputCol="2grams")
ngramFrame = twoGram.transform(ngramFrame)

In [11]:
threeGram = NGram(n=3, inputCol="opcodes", outputCol="3grams")
ngramFrame= threeGram.transform(ngramFrame)

In [12]:
fourGram = NGram(n=4, inputCol="opcodes", outputCol="4grams")
ngramFrame = fourGram.transform(ngramFrame)

In [13]:
twoGramRdd = ngramFrame.select("docId","2grams").rdd.map(tuple)
threeGramRdd =ngramFrame.select("docId","3grams").rdd.map(tuple)
fourGramRdd =ngramFrame.select("docId","4grams").rdd.map(tuple)

In [14]:
oneGramCounts = opcodesInDoc.flatMapValues(lambda x: x).map(lambda x: (x,1)).reduceByKey(add).map(lambda x: (x[0][0],(x[0][1],x[1])))

In [15]:
twoGramCounts = twoGramRdd.flatMapValues(lambda x: x).map(lambda x: (x,1)).reduceByKey(add).map(lambda x: (x[0][0],(x[0][1],x[1])))

In [16]:
threeGramCounts = threeGramRdd.flatMapValues(lambda x: x).map(lambda x: (x,1)).reduceByKey(add).map(lambda x: (x[0][0],(x[0][1],x[1])))

In [17]:
fourGramCounts = fourGramRdd.flatMapValues(lambda x: x).map(lambda x: (x,1)).reduceByKey(add).map(lambda x: (x[0][0],(x[0][1],x[1])))

In [18]:
print(twoGramCounts.take(10))

[(0, ('jge mov', 14)), (2, ('call and', 41)), (2, ('jl xor', 8)), (2, ('jz bts', 1)), (2, ('imul mov', 24)), (0, ('lea push', 64)), (2, ('push jb', 2)), (0, ('retn retn', 1)), (2, ('jge add', 11)), (0, ('not lea', 2))]


# Get all N-gram 

In [19]:
allTwoGrams = twoGramCounts.map(lambda x: (x[1][0],1)).reduceByKey(add).map(lambda x: x[0])

In [20]:
allThreeGrams = threeGramCounts.map(lambda x: (x[1][0],1)).reduceByKey(add).map(lambda x: x[0])

In [21]:
allFourGrams = fourGramCounts.map(lambda x: (x[1][0],1)).reduceByKey(add).map(lambda x: x[0])

# label wise N-gram


In [22]:
labels = sc.textFile("/home/vyom/UGA/DSP/Project2/data/train/y_small_train.txt")

In [23]:
labelRdd = labels.zipWithIndex().map(lambda x: (x[1],x[0]))

In [24]:
labelRdd = sc.parallelize(labelRdd.take(4))

In [25]:
labelWiseOneGramCounts = oneGramCounts.leftOuterJoin(labelRdd).map(lambda x: (x[1][1],(x[1][0],x[0])))

In [26]:
labelWisetTwoGramCounts = twoGramCounts.leftOuterJoin(labelRdd).map(lambda x: (x[1][1],(x[1][0],x[0])))

In [27]:
labelWiseThreeGramCounts = threeGramCounts.leftOuterJoin(labelRdd).map(lambda x: (x[1][1],(x[1][0],x[0])))

In [28]:
labelWiseFourGramCounts = fourGramCounts.leftOuterJoin(labelRdd).map(lambda x: (x[1][1],(x[1][0],x[0])))

In [30]:
print(labelWiseOneGramCounts.take(10))

[('6', (('fidiv', 2), 0)), ('6', (('fld1', 3), 0)), ('6', (('cld', 11), 0)), ('6', (('mov', 4338), 0)), ('6', (('call', 558), 0)), ('6', (('fabs', 1), 0)), ('6', (('in', 8), 0)), ('6', (('pop', 523), 0)), ('6', (('jmp', 226), 0)), ('6', (('fscale', 1), 0))]


# Adding smoothing

In [31]:
#labelWiseOneGramCounts.reduceByKey(add).collect()

# Extract Segments

In [32]:
segments = rdd1.zipWithIndex().map(lambda x: (x[1],x[0][1].splitlines())).map(lambda x: (x[0],[re.findall(r'\w+:?(?=:)',word) for word in x[1]])).flatMapValues(lambda x: x).map(lambda x: (x[0],x[1][0]))